In [21]:
import requests
import json

- Get all shops

In [ ]:
url = "https://www.foody.vn/__get/Place/HomeListPlace?page=1&lat=10.823099&lon=106.629664&count=12&districtId=&cateId=&cuisineId=&isReputation=&type=1"
all_shops_API = requests.get(url, headers={'X-Requested-With': 'XMLHttpRequest',
                                            'Cookie': 'floc=217'})
data = all_shops_API.text
dataJson = json.loads(data)
print(dataJson)

- Get menu of ONE shop: fải loop qua từng request_id trong cell code trên

In [23]:
# TEST
import pandas as pd

qbt_df = pd.read_csv("qbt.csv")

In [10]:
menu_df = pd.DataFrame(columns=['dish_type_id', 'dish_type_name'])
menu_dish_df = pd.DataFrame(columns=['dish_type_id', 'dish_id', 'dish_name', 'dish_description', 'dish_price_value', 'dish_total_like', 'dish_is_available'])
menu_dish_option_df = pd.DataFrame(columns=['dish_type_id', 'dish_id', 'option_id', 'option_min_select', 'option_max_select'])
menu_dish_option_item_df = pd.DataFrame(columns=['dish_type_id', 'dish_id', 'option_id', 'item_id', 'item_name', 'item_price'])

def get_delivery_dish(id):
    delivery_dish_API = requests.get(f"https://gappapi.deliverynow.vn/api/dish/get_delivery_dishes?request_id={id}&id_type=1", 
                                    headers={'x-foody-api-version': '1',
                                             'x-foody-app-type': '1004',
                                             'x-foody-client-language': 'vi',
                                             'x-foody-client-type': '1',
                                             'x-foody-client-version': '1',
                                             'X-Foody-Client-Id': 'cookies.__ondemand_sessionid'
                                        }
                                )
    data = delivery_dish_API.text
    dataJson = json.loads(data)
    menu_infos = dataJson['reply']['menu_infos']
    for menu_info in menu_infos:
        menu_df.loc[len(menu_df.index)] = [menu_info['dish_type_id'], menu_info['dish_type_name']]
        for dish in menu_info['dishes']:
                menu_dish_df.loc[len(menu_dish_df.index)] = [
                        menu_info['dish_type_id'],
                        dish['id'],
                        dish['name'],
                        dish['description'],
                        dish['price']['value'],
                        dish['total_like'],
                        dish['is_available']
                ]
                for option in dish['options']:
                        menu_dish_option_df.loc[len(menu_dish_option_df.index)] = [
                                menu_info['dish_type_id'],
                                dish['id'],
                                option['id'],
                                option['option_items']['min_select'],
                                option['option_items']['max_select']
                        ]
                        for item in option['option_items']['items']:
                                menu_dish_option_item_df.loc[len(menu_dish_option_item_df.index)] = [
                                        menu_info['dish_type_id'],
                                        dish['id'],
                                        option['id'],
                                        item['id'],
                                        item['name'],
                                        item['price']['value']
                                ]

for i in qbt_df['RestaurantId']:
    get_delivery_dish(i)

menu_df.to_csv('menu.csv', sep=',', encoding='utf-8')
menu_dish_df.to_csv('menu.dish.csv', sep=',', encoding='utf-8')
menu_dish_option_df.to_csv('menu.dish.option.csv', sep=',', encoding='utf-8')
menu_dish_option_item_df.to_csv('menu.dish.option.item.csv', sep=',', encoding='utf-8')

- get detail of ONE shop

In [ ]:
shop_df = pd.DataFrame(columns=['shop_id', 'name', 'short_description', 'city_id', 'district_id', 'address', 
                                'rating_total_review', 'rating_avg', 'rating_app_link',  
                                'service_fee', 'avg_price', 'min_order_value', 'min_charge', 'min_shipping_fee', 'min_price', 'max_price', 
                                'is_foody_delivery', 'week_days', 'start_time', 'end_time'])
shop_phone_df = pd.DataFrame(columns=['shop_id', 'phone'])
shop_category_df = pd.DataFrame(columns=['shop_id', 'category'])
shop_cuisine_df = pd.DataFrame(columns=['shop_id', 'cuisine'])
shop_promotion_df = pd.DataFrame(columns=['shop_id', 'promo_code', 'discount_amount', 'min_order_amount', 'max_discount_amount', 'apply_order','max_usage_time', 'expired'])


def get_detail(id):
        delivery_dish_API = requests.get(f"https://gappapi.deliverynow.vn/api/delivery/get_detail?request_id={id}&id_type=1", 
                                        headers={'x-foody-api-version': '1',
                                                'x-foody-app-type': '1004',
                                                'x-foody-client-language': 'vi',
                                                'x-foody-client-type': '1',
                                                'x-foody-client-version': '1',
                                                'X-Foody-Client-Id': 'cookies.__ondemand_sessionid'
                                                }
                                        )
        data = delivery_dish_API.text
        dataJson = json.loads(data)

        shop = dataJson['reply']['delivery_detail']


        shop_df.loc[len(shop_df.index)] = [
                shop['id'], shop['name'], shop['short_description'], shop['city_id'], shop['district_id'], shop['address'],
                shop['rating']['total_review'], shop['rating']['avg'], shop['rating']['app_link'],
                shop['delivery']['service_fee']['value'], shop['delivery']['avg_price']['value'], shop['delivery']['min_order_value']['value'],
                shop['delivery']['min_charge'], shop['delivery']['shipping_fee']['minimum_fee'],shop['price_range']['min_price'],
                shop['price_range']['max_price'],shop['delivery']['is_foody_delivery'],
                len(shop['delivery']['time']['week_days']), 
                shop['delivery']['time']['week_days'][0]['start_time'], 
                shop['delivery']['time']['week_days'][0]['end_time'] 
        ]
        
        for phone in shop['phones']:
                shop_phone_df.loc[len(shop_phone_df.index)] = [
                        shop['id'],
                        phone
                ]

        for category in shop['categories']:
                shop_category_df.loc[len(shop_category_df.index)] = [
                        shop['id'],
                        category
                ]                     

        for cuisine in shop['cuisines']:
                shop_cuisine_df.loc[len(shop_cuisine_df.index)] = [
                        shop['id'],
                        cuisine
                ]    

        for promotion in shop['delivery']['promotions']:
                shop_promotion_df.loc[len(shop_promotion_df.index)] = [
                        shop['id'], promotion['promo_code'], promotion['discount_amount'], promotion['min_order_amount'], 
                        promotion['max_discount_amount'], promotion['apply_order'], promotion['user_condition']['limit_per_user']['max_usage_time'], promotion['expired']
                ]


for i in qbt_df['RestaurantId']:
    get_detail(i)



shop_df.to_csv('shop.csv', sep=',', encoding='utf-8')
shop_phone_df.to_csv('shop.phone.csv', sep=',', encoding='utf-8')
shop_category_df.to_csv('shop.category.csv', sep=',', encoding='utf-8')
shop_cuisine_df.to_csv('shop.cuisine.csv', sep=',', encoding='utf-8')
shop_promotion_df.to_csv('shop.promotion.csv', sep=',', encoding='utf-8')